In [ ]:
# this code is meant to edit the example code.  



In [2]:
#Sensor_values_capture captures readings from steering sensor and throttle onto Arduino board

import sys
from pymata4 import pymata4
from pyfirmata import util
import time

In [3]:
#Must define port that the Arduino is plugged into such as 'COM4'

port='COM4'
arduinoUno = pymata4.Pymata4()

pymata4:  Version 1.15

Copyright (c) 2020 Alan Yorinks All Rights Reserved.

Opening all potential serial ports...

Waiting 4 seconds(arduino_wait) for Arduino devices to reset...

Searching for an Arduino configured with an arduino_instance = 1


RuntimeError: arduino_instance_id does not match a value on the boards.

In [4]:
steerSensorPin = 1
throttleSensorPin = 2



minSteerSensorValue = 10000
maxSteerSensorValue = -10000

minThrottleSensorValue = 10000
maxThrottleSensorValue = -10000

"""
#connect to autonomous vehicle
connectionResult = connect(port)

while connectionResult != "success":
    print("arduino failed to connect but trying again...")
    connectionResult = connect(port)

print("arduino connected!")

time.sleep(3)
"""
print("please turn the steering wheel and throttle multiple times from minimum to maximum states. Then close this program")

time.sleep(4)

while True:
    arduinoUno.set_pin_mode_analog_input(steerSensorPin)
    arduinoUno.set_pin_mode_analog_input(throttleSensorPin)
    currentSteeringReading, time_stamp = arduinoUno.analog_read(steerSensorPin)
    currentThrottleReading, time_stamp=arduinoUno.analog_read(throttleSensorPin)
    


    if currentSteeringReading != None or currentThrottleReading != None:

        if currentSteeringReading < minSteerSensorValue:
            minSteerSensorValue = currentSteeringReading
        elif currentSteeringReading > maxSteerSensorValue:
            maxSteerSensorValue = currentSteeringReading
        
        if currentThrottleReading < minThrottleSensorValue:
            minThrottleSensorValue = currentThrottleReading
        elif currentThrottleReading > maxThrottleSensorValue:
            maxThrottleSensorValue = currentThrottleReading

        print("minSteerSensorValue > " + str(minSteerSensorValue) + " maxSteerSensorValue > " + str(maxSteerSensorValue) + " minThrottleSensorValue > " + str(minThrottleSensorValue) + " maxThrottleSensorValue > " + str(maxThrottleSensorValue))

    else:
        
        if currentSteeringReading == None and currentThrottleReading == None:
            print("Steering and throttle sensors not recording. Check connections")
        elif currentSteeringReading == None:
            print("Steering sensor not recording.  Check connections")
        else:
            print("Throttle sensor not recording.  Check connections")
    
    time.sleep(0.1) 

please turn the steering wheel and throttle multiple times from minimum to maximum states. Then close this program


KeyboardInterrupt: 

In [5]:


mapValues = False  



In [6]:
'''

minThrottlePWM = 0 
maxThrottlePWM = 1.5 #real max 1

minSteeringPWM = 0 
maxSteeringPWM = 0.6 #real max 1

reverseSteering = -1 # make 1 to reverse steering
'''

'\n\nminThrottlePWM = 0 \nmaxThrottlePWM = 1.5 #real max 1\n\nminSteeringPWM = 0 \nmaxSteeringPWM = 0.6 #real max 1\n\nreverseSteering = -1 # make 1 to reverse steering\n'

In [7]:
#Variables for some items.  I changed the arduinoNano variable from the original build to arduinoUno variable because I happen
# to have an UNO.  The Nano should be big enough to handle this build but if you add more sensors, other inputs like brakes, 
#or robot arms, may need Uno.  Variable name does not matter if you use a Nano or Uno.



num_steps = 1600
rev_num_steps = -1600
steering_pins = [8,9]


arduinoUno.set_pin_mode_stepper(num_steps, steering_pins)
 
throttlePin = None

#This code block appears to confirm if Arduino is connected and communicating as well as sensors.

def connect(port):
    global arduinoUno, board, num_steps, rev_num_steps, steering_pins, throttlePin
    if arduinoUno == None:
        
        arduinoUno = Arduino(port)
        it = util.Iterator(arduinoUno)
        it.start()
        
        arduinoUno.set_pin_mode_analog_input(steerSensorPin)
        arduinoUno.set_pin_mode_analog_input(throttleSensorPin)
        

        arduinoUno.stepper_write(300, num_steps)

        return "success"
    else:
        return "error"

In [8]:
#currentSteeringReading, time_stamp = arduinoUno.analog_read(steerSensorPin)
#currentThrottleReading, time_stamp=arduinoUno.analog_read(throttleSensorPin)

In [9]:
#This code disconnects the arduino

def disconnect():
    global arduinoUno
    arduinoUno.digital[steering_pins].write(1)
    #board.digital[rightEnable].write(0)
    #board.digital[leftEnable].write(0)
    writeValue(0,0)
    arduinoUno.exit()

In [10]:
# Convert steering sensor to angle.  
# Based on some measurements, I believe the wheels traverse about 56 degrees.  COnverted to -23 degrees for left max 
#    and 23 degrees for right max

minSteerAngle=-23
maxSteerAngle=23

def SteerSensorValueToAngle(currentSteeringReading):
    minSteerAngle=-23
    maxSteerAngle=23
    currentSteerAngle=(currentSteeringReading-minSteerSensorValue)/(maxSteerSensorValue-minSteerSensorValue)*(maxSteerAngle-minSteerAngle)+minSteerAngle
    return currentSteerAngle


def ThrottleSensorValuetoPercentage(currentThrottleReading):
    currentThrottlePercentage=round((currentThrottleReading-minThrottleSensorValue)/(maxThrottleSensorValue-minThrottleSensorValue)*100,1)
    return currentThrottlePercentage

In [11]:
#handles all writing to disk
import os
import cv2
import datetime
import json
import math
import pandas as pd

dataName = None 
inImageWriter = None 
outImageWriter = None 
debugFile = None
data_dir = None

def setupWriter(debug):
    global dataName, inImageWriter, outImageWriter, debugFile, data_dir

    dataName = input("please type name of this run. This will be the name of the Data folder> ")

    current_directory = os.getcwd()
    data_dir = os.path.join(current_directory, dataName)
    try:
        os.mkdir(data_dir)
        print("Directory " , data_dir ,  " Created ") 
    except FileExistsError:
        print("Directory " , data_dir ,  " already exists")

    if debug:
        debug_dir = os.path.join(current_directory, "DEBUG_" + dataName)
        try:
            os.mkdir(debug_dir)
            print("Directory " , debug_dir ,  " Created ") 
        except FileExistsError:
            print("Directory " , debug_dir ,  " already exists")
        
        inImagePath = os.path.join(debug_dir, "InImage.avi")
        inImageWriter = cv2.VideoWriter(inImagePath,cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10,(512,256))
        outImagePath = os.path.join(debug_dir, "OutImage.avi")
        outImageWriter = cv2.VideoWriter(outImagePath,cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10,(512,256))

        debugFilePath = os.path.join(debug_dir, "DebugMessage.txt")
        debugFile = open(debugFilePath, "w")

#Train data 
def writeTrainData(angle, throttle, image, framecount):
    global record_df, record_list

    camName = str(framecount) + '_cam-image_array_.jpg'
    camPath = os.path.join(data_dir, camName)
    cv2.imwrite(camPath, image) 
    
    #record_row={'image_num': camName, 'angle': angle, 'throttle%': throttle}
    record_row=[camName, angle, throttle]

    record_list.append(record_row)
    return record_list
    

#Debugging messages
def writeInDebugImage(InImage):
    inImageWriter.write(InImage)

def writeOutDebugImage(OutImage):
    outImageWriter.write(OutImage)

def writeAngleAndThrottle(angle, throttle, framecount):
    debugFile.write(str(framecount) + " angle: " + str(angle) + " throttle%: " + str(throttle))
    debugFile.write("\n")

def writeControlMessage(message):
    debugFile.write(message)
    debugFile.write("\n")

In [12]:
setupWriter(False)

please type name of this run. This will be the name of the Data folder> test1
Directory  C:\Users\jeffa\SelfDrivingGoKartFromATVToy\test1  Created 


In [13]:
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

count=0
record_list = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
                
        # Display the resulting frame
        cv2.imshow('Frame',frame)
        
        currentSteeringReading, time_stamp = arduinoUno.analog_read(steerSensorPin)
        currentThrottleReading, time_stamp=arduinoUno.analog_read(throttleSensorPin)
        
        writeTrainData(SteerSensorValueToAngle(currentSteeringReading), ThrottleSensorValuetoPercentage(currentThrottleReading), frame, count)
        count+=1

 
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
 
        # Break the loop
    else: 
        break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()
record_df = pd.DataFrame(record_list)
record_path=os.path.join(data_dir, 'results.csv')
record_df.to_csv(record_path, index=False, header=False)

In [ ]:
'''
#use this to delete bad data in a directory



for f in os.listdir(data_dir):
    if not f.endswith(".jpg"):
        continue
        
    os.remove(os.path.join(data_dir, f))
'''

In [ ]:
"""

#This returns the mapped steering position

def mapSteerPosition (value, fromSource,  toSource,  fromTarget,  toTarget):
    mappedSteerPosition = (value - fromSource) / (toSource - fromSource) * (toTarget - fromTarget) + fromTarget
    if mappedSteerPosition > 1:
        mappedSteerPosition = 1
    elif mappedSteerPosition < -1:
        mappedSteerPosition = -1
    return mappedSteerPosition

def map(value, fromSource,  toSource,  fromTarget,  toTarget):
    return (value - fromSource) / (toSource - fromSource) * (toTarget - fromTarget) + fromTarget
    

#connect('/dev/ttyUSB0')
#while(True):
    #writeValue(1,0)
    #time.sleep(10)
    #disconnect()
"""

In [ ]:
"""

#This returns the value for the throttle



def mapThrottle (value, fromSource,  toSource,  fromTarget,  toTarget):
    mappedThottle = (value - fromSource) / (toSource - fromSource) * (toTarget - fromTarget) + fromTarget
    if mappedThottle > 1:
        mappedThottle = 1
    elif mappedThottle < 0:
        mappedThottle = 0
    return mappedThottle
    
"""

In [ ]:
"""

#This code actually reads the value from the throttle and sensor.  It returns the steering position and throttle position


def readValue():
    global arduinoUno
    if arduinoUno != None:
        steerPosition = arduinoUno.analog[steerSensorPin].read()  
        throttlePosition = arduinoUno.analog[throttleSensorPin].read() 
        #TODO translate value from min max to -1 1 and 0 1 for throttle
        if(steerPosition != None and throttlePosition != None):
            if(mapValues):
                mappedSteeringPosition = mapSteerPosition(steerPosition, minSteerSensorValue, maxSteerSensorValue, -1, 1)
                mappedThrottlePosition = mapThrottle(throttlePosition, minThrottleSensorValue, maxThrottleSensorValue, 0, 1)
                #print("readed steeringposition > " + str(mappedSteeringPosition) + " readed throttle " + str(mappedThrottlePosition))
                return((mappedSteeringPosition * reverseSteering,mappedThrottlePosition))
            else:
                #print("readed steeringposition > " + str(steerPosition) + " readed throttle " + str(throttlePosition))
                return((steerPosition,throttlePosition))
    else:
        return("error")

"""

In [ ]:
"""

#This writes values for steering and throttle based on sensor readings

def writeValue(left,right,throttle):
    global leftPWM, rightPWM, leftEnable, rightEnable, throttlePin

    mappedLeft = 0
    mappedRight = 0

    if left > 0:
        rightPWM.write(0)
        mappedLeft = round(map(left, -1, 1, minSteeringPWM, maxSteeringPWM),3)
        leftPWM.write(mappedLeft)
    elif right > 0:
        leftPWM.write(0)
        mappedRight = round(map(right, -1, 1, minSteeringPWM, maxSteeringPWM),3)
        rightPWM.write(mappedRight)
    else:
        print("left right commando incorreect")

    mappedThrottle = round(map(throttle, 0, 1, minThrottlePWM, maxThrottlePWM),3)
    #print("writing values left> " + str(mappedLeft) + " right> " + str(mappedRight) + " throttle> " + str(mappedThrottle))
    throttlePin.write(mappedThrottle)
"""